## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-30-11-53-06 +0000,bbc,'It's not jubilation... ' On the ground at Lab...,https://www.bbc.com/news/videos/cp8w4xwnendo?a...
1,2025-09-30-11-49-55 +0000,bbc,Third of workforce to go at Lindsey Oil Refinery,https://www.bbc.com/news/articles/ckg6e78rme2o...
2,2025-09-30-11-46-23 +0000,nyt,Memphis Braces for the National Guard,https://www.nytimes.com/2025/09/30/us/politics...
3,2025-09-30-11-39-24 +0000,nyt,U.S. Deports Planeload of Iranians After Deal ...,https://www.nytimes.com/2025/09/30/world/middl...
4,2025-09-30-11-37-50 +0000,nyt,Dozens of Students Are Missing After School Co...,https://www.nytimes.com/2025/09/30/world/asia/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2313/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
51,trump,60
87,shutdown,20
86,government,19
103,new,18
60,gaza,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
113,2025-09-30-03-05-00 +0000,wsj,Top Democrats made their case to Trump for res...,https://www.wsj.com/politics/policy/government...,148
197,2025-09-29-21-14-29 +0000,nypost,Trump unveils 20-point Gaza cease-fire plan wi...,https://nypost.com/2025/09/29/us-news/trump-un...,146
172,2025-09-29-22-29-55 +0000,nypost,Ex-UK PM Tony Blair to help run Gaza under Tru...,https://nypost.com/2025/09/29/us-news/ex-uk-pm...,123
91,2025-09-30-06-50-20 +0000,nypost,Trump mocks Chuck Schumer and Hakeem Jeffries ...,https://nypost.com/2025/09/30/us-news/trump-mo...,123
209,2025-09-29-20-46-24 +0000,bbc,Watch: Trump and Netanyahu outline peace plan ...,https://www.bbc.com/news/videos/c24r8zj1mnro?a...,117


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
113,148,2025-09-30-03-05-00 +0000,wsj,Top Democrats made their case to Trump for res...,https://www.wsj.com/politics/policy/government...
197,86,2025-09-29-21-14-29 +0000,nypost,Trump unveils 20-point Gaza cease-fire plan wi...,https://nypost.com/2025/09/29/us-news/trump-un...
50,71,2025-09-30-09-30-00 +0000,wsj,With roughly a month left in the New York City...,https://www.wsj.com/politics/elections/cuomo-p...
223,44,2025-09-29-19-32-00 +0000,wsj,The clock is ticking down to an Oct. 1 deadlin...,https://www.wsj.com/politics/policy/government...
49,43,2025-09-30-09-30-00 +0000,wsj,A government shutdown would likely mean federa...,https://www.wsj.com/politics/policy/how-govern...
231,40,2025-09-29-18-55-45 +0000,nypost,Frank founder Charlie Javice sobs in court as ...,https://nypost.com/2025/09/29/business/charlie...
149,35,2025-09-30-00-01-00 +0000,wsj,American Eagle Outfitters’ chief executive had...,https://www.wsj.com/business/retail/you-cant-r...
84,35,2025-09-30-07-40-13 +0000,nypost,Woman disrobes at California school board meet...,https://nypost.com/2025/09/30/us-news/woman-di...
206,32,2025-09-29-20-53-00 +0000,wsj,The Trump administration said it is cracking d...,https://www.wsj.com/politics/national-security...
217,31,2025-09-29-20-03-08 +0000,nyt,"Aron Bell, Last Brother of a World War II Resi...",https://www.nytimes.com/2025/09/26/us/aron-bel...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
